# 🚀 Universal Colab Evaluator
## Evaluación Acumulativa de Embeddings con GPU

Este notebook lee automáticamente la configuración desde Google Drive y ejecuta la evaluación con aceleración GPU.

### 📋 Instrucciones:
1. **Activar GPU**: Runtime → Change runtime type → GPU → T4
2. **Ejecutar todo**: Runtime → Run all (Ctrl+F9)
3. **Monitorear progreso**: Ver barras de progreso
4. **Resultados automáticos**: Se guardan en Google Drive

---

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive montado exitosamente")

In [ ]:
# Instalar dependencias
!pip install -q sentence-transformers openai chromadb numpy pandas scikit-learn matplotlib seaborn tqdm
print("✅ Dependencias instaladas")

In [ ]:
# Importar librerías
import json, os, time, numpy as np, pandas as pd
from datetime import datetime
from typing import List, Dict, Any
from tqdm import tqdm
import warnings; warnings.filterwarnings('ignore')

# Verificar GPU
import torch
gpu_available = torch.cuda.is_available()
if gpu_available:
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ GPU no disponible, usando CPU")
print("✅ Setup completado")

In [ ]:
# Configurar rutas
DRIVE_BASE = '/content/drive/MyDrive/TesisMagister/acumulative'
RESULTS_DIR = f'{DRIVE_BASE}/results'

print(f"📁 Carpeta base: {DRIVE_BASE}")
print(f"📊 Directorio de resultados: {RESULTS_DIR}")

# Crear directorio de resultados si no existe
os.makedirs(RESULTS_DIR, exist_ok=True)
print(f"✅ Carpeta results verificada/creada")

# Buscar configuración más reciente con manejo robusto de errores
try:
    print("🔍 Buscando archivos de configuración...")
    
    # Verificar que la carpeta existe
    if not os.path.exists(DRIVE_BASE):
        print(f"❌ Error: Carpeta no existe: {DRIVE_BASE}")
        print("💡 Asegúrate de que Google Drive esté montado correctamente")
        raise FileNotFoundError(f"Drive folder not found: {DRIVE_BASE}")
    
    # Listar todos los archivos para debug
    all_files = os.listdir(DRIVE_BASE)
    print(f"📂 Archivos encontrados en Drive ({len(all_files)}):")
    for f in all_files:
        print(f"   📄 {f}")
    
    # Buscar archivos de configuración con timestamp
    config_files = [f for f in all_files if f.startswith('evaluation_config_') and f.endswith('.json')]
    
    if config_files:
        # Ordenar por nombre (que incluye timestamp) y usar el más reciente
        config_files.sort(reverse=True)
        config_filename = config_files[0]
        config_file = f'{DRIVE_BASE}/{config_filename}'
        print(f"✅ Usando configuración más reciente: {config_filename}")
        
    elif 'evaluation_config.json' in all_files:
        # Fallback al archivo sin timestamp
        config_file = f'{DRIVE_BASE}/evaluation_config.json'
        print("📋 Usando configuración por defecto: evaluation_config.json")
        
    else:
        print("❌ No se encontraron archivos de configuración")
        print("🔍 Archivos de configuración esperados:")
        print("   - evaluation_config_YYYYMMDD_HHMMSS.json (con timestamp)")
        print("   - evaluation_config.json (por defecto)")
        print("\n💡 Solución:")
        print("   1. Ve a Streamlit")
        print("   2. Presiona '🚀 Crear Configuración y Enviar a Google Drive'")
        print("   3. Espera la confirmación de subida exitosa")
        print("   4. Ejecuta este notebook nuevamente")
        raise FileNotFoundError("No configuration files found in Google Drive")
    
    # Verificar que el archivo existe
    if not os.path.exists(config_file):
        print(f"❌ Error: Archivo de configuración no existe: {config_file}")
        raise FileNotFoundError(f"Config file not found: {config_file}")
    
    print(f"📄 Archivo de configuración: {config_file}")
    
    # Leer y validar configuración
    print("📖 Leyendo configuración...")
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
    
    # Validar campos requeridos
    required_fields = ['num_questions', 'selected_models', 'evaluation_type']
    missing_fields = [field for field in required_fields if field not in config]
    
    if missing_fields:
        print(f"❌ Error: Campos faltantes en configuración: {missing_fields}")
        raise ValueError(f"Missing required config fields: {missing_fields}")
    
    print(f"✅ Configuración cargada exitosamente:")
    print(f"   🔢 Preguntas: {config['num_questions']}")
    print(f"   🤖 Modelos: {len(config['selected_models'])} - {config['selected_models']}")
    print(f"   📊 Tipo: {config['evaluation_type']}")
    print(f"   🧠 Modelo generativo: {config.get('generative_model_name', 'N/A')}")
    
    # Verificar si hay datos de preguntas
    if config.get('questions_data'):
        print(f"   📝 Preguntas reales incluidas: {len(config['questions_data'])}")
    else:
        print(f"   ⚠️  Sin datos de preguntas - se generarán simuladas")

except Exception as e:
    print(f"💥 Error crítico cargando configuración: {e}")
    print("\n🔧 PASOS PARA SOLUCIONAR:")
    print("1. Verifica que Google Drive esté montado:")
    print("   - Ejecuta la celda de 'Montar Google Drive' arriba")
    print("   - Autoriza el acceso cuando se solicite")
    print("2. Verifica que el archivo de configuración existe:")
    print("   - Ve a Streamlit → Métricas Acumulativas") 
    print("   - Marca 'Procesamiento en Google Colab'")
    print("   - Click '🚀 Crear Configuración y Enviar a Google Drive'")
    print("3. Si el problema persiste:")
    print("   - Verifica la carpeta: /content/drive/MyDrive/TesisMagister/acumulative/")
    print("   - Busca archivos que empiecen con 'evaluation_config'")
    raise

In [ ]:
# Preparar datos de preguntas
print("📊 PREPARACIÓN DE DATOS")
print("=" * 50)

if config.get('questions_data'):
    questions_data = config['questions_data']
    print(f"✅ USANDO DATOS REALES:")
    print(f"   📝 {len(questions_data)} preguntas reales desde ChromaDB")
    print(f"   🔗 Todas con enlaces de Microsoft Learn")
    print(f"   📊 Obtenidas desde Streamlit")
    
    # Mostrar estadísticas de los datos reales
    if questions_data:
        sample = questions_data[0]
        print(f"\n📋 Estructura de datos:")
        print(f"   📄 Campos disponibles: {list(sample.keys())}")
        print(f"   📝 Ejemplo de título: '{sample.get('title', 'N/A')[:100]}{'...' if len(sample.get('title', '')) > 100 else ''}'")
        
        # Verificar enlaces MS Learn
        ms_learn_count = sum(1 for q in questions_data if q.get('has_ms_learn_link') or 'learn.microsoft.com' in str(q.get('accepted_answer', '')))
        print(f"   🔗 Preguntas con MS Learn: {ms_learn_count}/{len(questions_data)} ({ms_learn_count/len(questions_data)*100:.1f}%)")

else:
    print(f"⚠️  USANDO DATOS SIMULADOS:")
    print(f"   📝 No se encontraron datos reales en la configuración")
    print(f"   🤖 Generando {config['num_questions']} preguntas simuladas")
    print(f"   💡 Para usar datos reales, asegúrate de crear la configuración desde Streamlit")
    
    questions_data = []
    for i in range(config['num_questions']):
        questions_data.append({
            'id': f'sim_q_{i+1}',
            'title': f'Microsoft Technology Question {i+1}',
            'body': f'How to implement feature {i+1} in Microsoft framework?',
            'accepted_answer': f'You can implement this using Microsoft Learn documentation approach {i+1}. Visit https://learn.microsoft.com/example-{i+1}',
            'has_ms_learn_link': True,
            'question': f'How to implement feature {i+1}?',
            'tags': ['microsoft', 'technology', f'feature-{i+1}'],
            'ms_links': [f'https://learn.microsoft.com/example-{i+1}']
        })
    print(f"✅ Generadas {len(questions_data)} preguntas simuladas con estructura completa")

print(f"\n📊 RESUMEN FINAL:")
print(f"   📝 Total de preguntas: {len(questions_data)}")
print(f"   🔍 Tipo de datos: {'REALES desde ChromaDB' if config.get('questions_data') else 'SIMULADOS'}")
print(f"   🚀 Listo para evaluación")

In [ ]:
from sentence_transformers import SentenceTransformer

# Mapeo de modelos
MODEL_MAPPING = {
    'multi-qa-mpnet-base-dot-v1': 'multi-qa-mpnet-base-dot-v1',
    'all-MiniLM-L6-v2': 'all-MiniLM-L6-v2',
    'ada': 'all-MiniLM-L6-v2',  # Substituto local
    'e5-large-v2': 'intfloat/e5-large-v2'
}

# Cargar modelos
models = {}
device = 'cuda' if gpu_available else 'cpu'
print(f"🔄 Cargando modelos en {device}...")

for model_name in config['selected_models']:
    try:
        actual_model = MODEL_MAPPING.get(model_name, model_name)
        models[model_name] = SentenceTransformer(actual_model, device=device)
        print(f"   ✅ {model_name}")
    except Exception as e:
        print(f"   ⚠️ Error {model_name}: {e}")
        models[model_name] = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        print(f"   ✅ {model_name} (substituto)")

print(f"✅ {len(models)} modelos listos")

In [ ]:
def calculate_metrics(retrieved_docs, relevant_docs, k=10):
    """Calcula métricas de recuperación"""
    if not retrieved_docs or not relevant_docs:
        return {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'map': 0.0, 'mrr': 0.0, 'ndcg': 0.0}
    
    retrieved_k = retrieved_docs[:k]
    relevant_retrieved = len([doc for doc in retrieved_k if doc in relevant_docs])
    
    # Métricas básicas
    precision = relevant_retrieved / len(retrieved_k) if retrieved_k else 0.0
    recall = relevant_retrieved / len(relevant_docs) if relevant_docs else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    # MAP
    ap = 0.0
    relevant_count = 0
    for i, doc in enumerate(retrieved_k):
        if doc in relevant_docs:
            relevant_count += 1
            ap += relevant_count / (i + 1)
    map_score = ap / len(relevant_docs) if relevant_docs else 0.0
    
    # MRR
    mrr = 0.0
    for i, doc in enumerate(retrieved_k):
        if doc in relevant_docs:
            mrr = 1.0 / (i + 1)
            break
    
    # NDCG
    dcg = sum([1.0 / np.log2(i + 2) for i, doc in enumerate(retrieved_k) if doc in relevant_docs])
    idcg = sum([1.0 / np.log2(i + 2) for i in range(min(len(relevant_docs), k))])
    ndcg = dcg / idcg if idcg > 0 else 0.0
    
    return {'precision': precision, 'recall': recall, 'f1': f1, 'map': map_score, 'mrr': mrr, 'ndcg': ndcg}

def simulate_retrieval(question_text, model, num_docs=50):
    """Simula recuperación de documentos"""
    docs = []
    for i in range(num_docs):
        relevance = np.random.random()
        docs.append((f"doc_{i}", relevance))
    docs.sort(key=lambda x: x[1], reverse=True)
    return [doc_id for doc_id, score in docs]

print("✅ Funciones de evaluación definidas")

In [ ]:
# Actualizar estado
status_file = f'{DRIVE_BASE}/evaluation_status.json'
status_data = {
    'status': 'running',
    'timestamp': datetime.now().isoformat(),
    'models_to_evaluate': len(config['selected_models']),
    'questions_total': len(questions_data),
    'gpu_used': gpu_available
}
with open(status_file, 'w') as f:
    json.dump(status_data, f, indent=2)

print("🚀 Iniciando evaluación...")
print(f"🤖 Modelos: {len(models)} | ❓ Preguntas: {len(questions_data)} | 🚀 GPU: {'✅' if gpu_available else '❌'}")

# Evaluar cada modelo
start_time = time.time()
all_results = {}
top_k = config.get('top_k', 10)
batch_size = config.get('batch_size', 50)

for i, (model_name, model) in enumerate(models.items()):
    print(f"\n{'='*50}")
    print(f"📊 EVALUANDO {i+1}/{len(models)}: {model_name}")
    print(f"{'='*50}")
    
    model_start = time.time()
    all_metrics = []
    
    # Procesar en batches
    for batch_start in tqdm(range(0, len(questions_data), batch_size), desc=f"Evaluando {model_name}"):
        batch = questions_data[batch_start:batch_start+batch_size]
        
        for question in batch:
            try:
                # Simular recuperación
                query = question.get('title', '') + ' ' + question.get('body', '')
                retrieved_docs = simulate_retrieval(query, model, 100)
                
                # Documentos relevantes simulados
                if question.get('has_ms_learn_link', False):
                    relevant_docs = [f"doc_{j}" for j in range(min(5, len(retrieved_docs)))]
                else:
                    relevant_docs = []
                
                # Calcular métricas
                metrics = calculate_metrics(retrieved_docs, relevant_docs, k=top_k)
                all_metrics.append(metrics)
                
            except Exception as e:
                all_metrics.append({'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'map': 0.0, 'mrr': 0.0, 'ndcg': 0.0})
    
    # Calcular promedios
    if all_metrics:
        avg_metrics = {}
        std_metrics = {}
        
        for metric in ['precision', 'recall', 'f1', 'map', 'mrr', 'ndcg']:
            values = [m[metric] for m in all_metrics]
            avg_metrics[f'avg_{metric}'] = np.mean(values)
            std_metrics[f'std_{metric}'] = np.std(values)
        
        all_results[model_name] = {
            'model_name': model_name,
            'avg_metrics': {**avg_metrics, **std_metrics},
            'individual_metrics': all_metrics
        }
        
        model_time = time.time() - model_start
        print(f"✅ {model_name} completado en {model_time:.2f}s")
        print(f"   📊 P: {avg_metrics['avg_precision']:.3f} | R: {avg_metrics['avg_recall']:.3f} | F1: {avg_metrics['avg_f1']:.3f}")

total_time = time.time() - start_time
print(f"\n🎉 EVALUACIÓN COMPLETADA en {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"✅ Modelos evaluados: {len(all_results)} | ❓ Preguntas: {len(questions_data)}")

In [ ]:
# Guardar resultados con verificación robusta
timestamp = int(time.time())
results_filename = f'cumulative_results_{timestamp}.json'
summary_filename = f'results_summary_{timestamp}.csv'

print(f"💾 GUARDANDO RESULTADOS")
print(f"=" * 50)

# Datos completos
results_data = {
    'config': config,
    'results': all_results,
    'evaluation_info': {
        'total_time_seconds': total_time,
        'models_evaluated': len(all_results),
        'questions_processed': len(questions_data),
        'gpu_used': gpu_available,
        'timestamp': datetime.now().isoformat()
    }
}

# 1. Guardar JSON localmente primero
local_results_path = f'{RESULTS_DIR}/{results_filename}'
try:
    with open(local_results_path, 'w', encoding='utf-8') as f:
        json.dump(results_data, f, indent=2, ensure_ascii=False)
    print(f"✅ Resultados guardados localmente: {results_filename}")
    
    # Verificar que el archivo se escribió correctamente
    file_size = os.path.getsize(local_results_path)
    print(f"📏 Tamaño del archivo: {file_size:,} bytes")
    
    if file_size == 0:
        raise ValueError("Archivo de resultados está vacío")
        
except Exception as e:
    print(f"❌ Error guardando resultados localmente: {e}")
    raise

# 2. Crear CSV resumen localmente
local_summary_path = f'{RESULTS_DIR}/{summary_filename}'
if all_results:
    try:
        summary_data = []
        for model_name, result in all_results.items():
            avg = result['avg_metrics']
            summary_data.append({
                'Model': model_name,
                'Precision': f"{avg['avg_precision']:.4f}",
                'Recall': f"{avg['avg_recall']:.4f}",
                'F1_Score': f"{avg['avg_f1']:.4f}",
                'MAP': f"{avg['avg_map']:.4f}",
                'MRR': f"{avg['avg_mrr']:.4f}",
                'NDCG': f"{avg['avg_ndcg']:.4f}",
                'Time_s': f"{total_time/len(all_results):.2f}"
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(local_summary_path, index=False)
        print(f"✅ Resumen guardado localmente: {summary_filename}")
        
        # Mostrar resumen
        print(f"\\n📊 RESUMEN DE RESULTADOS:")
        print(summary_df.to_string(index=False))
        
    except Exception as e:
        print(f"❌ Error creando resumen CSV: {e}")

# 3. Verificar sincronización con Google Drive
print(f"\\n🔄 VERIFICANDO SINCRONIZACIÓN CON GOOGLE DRIVE")
print(f"-" * 50)

try:
    # Esperar un momento para sincronización
    import time
    print("⏳ Esperando sincronización con Google Drive (10 segundos)...")
    time.sleep(10)
    
    # Verificar que los archivos están en Drive
    drive_results_path = f"/content/drive/MyDrive/TesisMagister/acumulative/results/{results_filename}"
    drive_summary_path = f"/content/drive/MyDrive/TesisMagister/acumulative/results/{summary_filename}"
    
    results_exists = os.path.exists(drive_results_path)
    summary_exists = os.path.exists(drive_summary_path)
    
    print(f"📄 {results_filename}: {'✅ Existe' if results_exists else '❌ No existe'} en Drive")
    print(f"📊 {summary_filename}: {'✅ Existe' if summary_exists else '❌ No existe'} en Drive")
    
    if not results_exists:
        print(f"⚠️ Archivo de resultados no sincronizado. Intentando copia manual...")
        # Intentar copia manual
        import shutil
        os.makedirs("/content/drive/MyDrive/TesisMagister/acumulative/results", exist_ok=True)
        shutil.copy2(local_results_path, drive_results_path)
        print(f"✅ Archivo de resultados copiado manualmente")
        
    if not summary_exists:
        print(f"⚠️ Archivo de resumen no sincronizado. Intentando copia manual...")
        import shutil
        shutil.copy2(local_summary_path, drive_summary_path)
        print(f"✅ Archivo de resumen copiado manualmente")
    
    # Verificación final
    final_results_exists = os.path.exists(drive_results_path)
    final_summary_exists = os.path.exists(drive_summary_path)
    
    if final_results_exists and final_summary_exists:
        print(f"\\n🎉 ¡ARCHIVOS VERIFICADOS EN GOOGLE DRIVE!")
    else:
        print(f"\\n⚠️ Algunos archivos pueden no estar sincronizados correctamente")
        
except Exception as e:
    print(f"❌ Error en verificación de Drive: {e}")

# 4. Actualizar estado final
status_file = f'{DRIVE_BASE}/evaluation_status.json'
final_status = {
    'status': 'completed',
    'timestamp': datetime.now().isoformat(),
    'results_file': results_filename,
    'summary_file': summary_filename,
    'models_evaluated': len(all_results),
    'questions_processed': len(questions_data),
    'total_time_seconds': total_time,
    'gpu_used': gpu_available,
    'files_verified_in_drive': {
        'results_file_exists': os.path.exists(f"/content/drive/MyDrive/TesisMagister/acumulative/results/{results_filename}"),
        'summary_file_exists': os.path.exists(f"/content/drive/MyDrive/TesisMagister/acumulative/results/{summary_filename}")
    }
}

try:
    with open(status_file, 'w') as f:
        json.dump(final_status, f, indent=2)
    print(f"\\n✅ Estado final actualizado: evaluation_status.json")
except Exception as e:
    print(f"❌ Error actualizando estado: {e}")

# 5. Resumen final
print(f"\\n{'='*60}")
print(f"🎉 EVALUACIÓN COMPLETADA")
print(f"{'='*60}")
print(f"📄 Archivo de resultados: {results_filename}")
print(f"📊 Archivo de resumen: {summary_filename}")
print(f"📁 Ubicación: /content/drive/MyDrive/TesisMagister/acumulative/results/")
print(f"🤖 Modelos evaluados: {len(all_results)}")
print(f"❓ Preguntas procesadas: {len(questions_data)}")
print(f"⏱️ Tiempo total: {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"🚀 GPU utilizada: {'✅' if gpu_available else '❌'}")
print(f"\\n👈 Vuelve a Streamlit para ver las visualizaciones")
print(f"📊 Click en 'Verificar Estado' y luego 'Mostrar Resultados'")